In [ ]:
from bloqade import squin
from kirin.dialects.ilist import IList
import numpy as np
import bloqade.stim
import bloqade.tsim

In [ ]:
def MSD_encoding(theta, phi,basis="z"):

    @squin.kernel
    def parameterized_MSD_encoding():
        q = squin.qalloc(7)  # allocate qubits
        squin.u3(theta,phi,0,q[6])
        for i in range(6):
            squin.sqrt_y_adj(q[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q[1], q[2])
        squin.cz(q[3], q[4])
        squin.cz(q[5], q[6])
        squin.sqrt_y(q[6])
        squin.cz(q[0],q[3])
        squin.cz(q[2],q[5])
        squin.cz(q[4],q[6])
        for i in range(5):
            squin.sqrt_y(q[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q[0],q[1])
        squin.cz(q[2],q[3])
        squin.cz(q[4],q[5])
        squin.sqrt_y(q[1])
        squin.sqrt_y(q[2])
        squin.sqrt_y(q[4])
    
    return parameterized_MSD_encoding

In [ ]:
def error_detection(theta, phi):
    @squin.kernel
    MSD_enc = MSD_encoding(theta, phi)
    z_ancilla = MSD_encoding((np.pi)/2, 0) # Injected state is |+>

    for i in range(7):
        squin.cx(MSD_enc.q[i], z_ancilla.q[i])
        squin.measure(z_ancilla.q[i])
    
    x_ancilla = MSD_encoding(0, 0) # Injected state is |0>
    for i in range(7):
        squin.cx(x_ancilla.q[i], MSD_enc.q[i])
        squin.h(x_ancilla.q[i])
        squin.measure(x_ancilla.q[i])